In [ ]:
from datetime import datetime, timedelta
from datetime import time as dt
from pytz import timezone

from fastapi import FastAPI, WebSocket, Request, WebSocketDisconnect
from fastapi.templating import Jinja2Templates

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

import numpy as np
import pymysql
import base64
import requests
import locale
import wave
import json
import os
import re

db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '1234567890',
    'database': 'CounselingVoiceBot'
}

db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '1234567890',
    'database': 'seniorcare'
}


# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) # Load the .env file

# Initialize the OpenAI client with the API key
openai_client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],  # Retrieves API key from environment variables
)

context = []
context_string = []
context_counter = 0

phone_id = 51



In [ ]:


class UserInfo:
    def __init__(self, db_config):
        self.db_config = db_config

    def get_total_userinfo_from_db(self, phone_id):
        try:
            connection = pymysql.connect(**self.db_config)
            cursor = connection.cursor()
            
            query = """SELECT 
                            user.phone_id, 
                            user.username, 
                            user.userid, 
                            user.usersex, 
                            user.userage,
                            healthinfo.disease,
                            alarm.casual_alarm_time, 
                            healthinfo.medication,
                            healthinfo.injection, 
                            healthinfo.healthissue,
                            summarization.conversation_start,
                            summarization.summary
                        FROM 
                            user 
                        JOIN 
                            alarm ON user.phone_id = alarm.phone_id 
                        JOIN 
                            healthinfo ON user.phone_id = healthinfo.phone_id
                        JOIN 
                            summarization ON user.phone_id = summarization.phone_id
                        WHERE 
                            user.phone_id = %s
                            AND summarization.session_id = (
                                SELECT session_id 
                                FROM summarization 
                                WHERE phone_id = %s 
                                ORDER BY summ_created_at DESC 
                                LIMIT 1
                            );"""
            values = (phone_id,phone_id)
            
            cursor.execute(query, values)
            total_userinfo_db = cursor.fetchall()

        except pymysql.Error as error:
            print(f"Error while connecting to MySQL: {error}")
            total_userinfo_db = None

        finally:
            cursor.close()
            connection.close()
            
        return total_userinfo_db

    def parse_time(self, time_str):
        return datetime.strptime(time_str, "%H:%M").time()

    def convert_to_korean_time(self, time_obj):
        korean_time = (datetime.combine(datetime.today(), time_obj) + timedelta(hours=9)).time()
        return korean_time

    def process_alarms(self, alarm_json_str):
        alarms = json.loads(alarm_json_str)
        if isinstance(alarms, list):
            for alarm in alarms:
                for key, value in alarm.items():
                    if isinstance(value, list):
                        alarm[key] = [self.convert_to_korean_time(self.parse_time(t)) for t in value]
                    elif isinstance(value, dict):
                        for med, times in value.items():
                            value[med] = [self.convert_to_korean_time(self.parse_time(t)) for t in times]
        return alarms
    
    def get_medication_name(self, medication_json_str):
        medications = json.loads(medication_json_str)
        user_medication = []
        if isinstance(medications, list):
            for med_dict in medications:
                for med in med_dict.keys():
                    user_medication.append(med)
        return user_medication
    
    def get_injection_name(self, injection_json_str):
        injections = json.loads(injection_json_str)
        user_injection = []
        if isinstance(injections, list):
            for inj_dict in injections:
                for inj in inj_dict.keys():
                    user_injection.append(inj)
        return user_injection
            

    def get_user_info(self, phone_id):
        data = self.get_total_userinfo_from_db(phone_id)
        if not data:
            return None
        
        user_info = {
            "phone_id": data[0][0],
            "username": data[0][1],
            "userid": data[0][2],
            "usersex": data[0][3],
            "userage": data[0][4],
            "disease": data[0][5],
            "casualalarm": self.process_alarms(data[0][6]),
            "medication": self.get_medication_name(data[0][7]),
            "health_med_alarm": self.process_alarms(data[0][7]),
            "injection": self.get_injection_name(data[0][8]),
            "health_inj_alarm": self.process_alarms(data[0][8]),
            "healthissue": data[0][9],
            "prev_conversation_start": data[0][10],
            "prev_summary": data[0][11]
        }
        
        return user_info
        


In [35]:

class UserSession:
    def __init__(self, phone_id, data):
        self.phone_id = phone_id
        self.data = data
        self.time_delta = timedelta(minutes=3)
        self.current_time = datetime.now(timezone('Asia/Seoul')).time()
        self.close_time, self.close_key = self.check_times()
        self.session_id = self.generate_session_id()
        self.sessiontime = self.get_sessiontime()
        self.current_timearea = self.get_current_timearea()

    def generate_session_id(self):
        current_time_str = datetime.now(timezone('Asia/Seoul')).strftime('%Y%m%d%H%M%S')
        session_id = f"{current_time_str}_{self.phone_id}"
        return session_id

    def is_within_time_range(self, target_time, reference_time, delta):
        reference_datetime = datetime.combine(datetime.today(), reference_time)
        target_datetime = datetime.combine(datetime.today(), target_time)
        return abs(reference_datetime - target_datetime) <= delta

    def check_times(self):
        close_time = None
        close_key = None
        for key, value in self.data.items():
            if isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        for sub_key, times in item.items():
                            for t in times:
                                if self.is_within_time_range(t, self.current_time, self.time_delta):
                                    close_time = t
                                    close_key = key
                    elif isinstance(item, list):
                        for t in item:
                            if self.is_within_time_range(t, self.current_time, self.time_delta):
                                close_time = t
                                close_key = key
        return close_time, close_key

    def get_sessiontime(self):
        session_datetime = datetime.strptime(self.session_id.split('_')[0], '%Y%m%d%H%M%S')
        sessiontime = session_datetime.time().replace(second=0, microsecond=0)
        sessiontime = dt(sessiontime.hour, sessiontime.minute)  # Only keep hour and minute
        return sessiontime
    
    def get_current_timearea(self):
        if datetime.strptime("06:00", "%H:%M").time() <= self.sessiontime <= datetime.strptime("11:59", "%H:%M").time():
            current_timearea = "아침"
        elif datetime.strptime("12:00", "%H:%M").time() <= self.sessiontime <= datetime.strptime("17:59", "%H:%M").time():
            current_timearea = "점심"
        elif datetime.strptime("18:00", "%H:%M").time() <= self.sessiontime <= datetime.strptime("23:59", "%H:%M").time():
            current_timearea = "저녁"
        else:
            current_timearea = "새벽"
        return current_timearea

In [36]:
def get_transcript(file_path):
    # Open the audio file in binary read mode
    audio_file = open(file_path, "rb")
    
    # Use the OpenAI Whisper model to transcribe the audio
    transcript = openai_client.audio.transcriptions.create(
        model="whisper-1",           # Specifies the Whisper model to use
        file=audio_file,             # Passes the audio file to the API
        response_format="text"       # Requests the transcription in text format
    )

    # Return the transcription
    return transcript

In [ ]:


def save_context_to_db(role, content, conversation_start=None, conv_model=None):
    global context_counter
    context_counter += 1
    global session_id
    global phone_id
    global username
    current_time = datetime.now(timezone('Asia/Seoul')).strftime('%Y-%m-%d %H:%M:%S')

    try:
        
        connection = pymysql.connect(**db_config)
        cursor = connection.cursor()

        
        query = "INSERT INTO context (session_id, unique_number, created_at, phone_id, username, conversation_start, model, role, content) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (session_id, context_counter, current_time, phone_id, username, conversation_start, conv_model, role, content)

        
        cursor.execute(query, values)
        connection.commit()

        print("Context saved to the database.")

    except pymysql.Error as error:
        print(f"Error while connecting to MySQL: {error}")

    finally:
        cursor.close()
        connection.close()
        
    if role == "user":
        userevenno = context_counter
        return userevenno

In [ ]:
def save_audiodir_to_context(audiodir, session_id, userevenno):
    
    try:
        
        connection = pymysql.connect(**db_config)
        cursor = connection.cursor()

        
        query = """UPDATE context
                    SET audio_file_dir = %s
                    WHERE session_id = %s AND unique_number = %s AND role = 'user';"""
        values = (audiodir, session_id, userevenno)

        
        cursor.execute(query, values)
        connection.commit()

        print("Audio directory saved to the database.")

    except pymysql.Error as error:
        print(f"Error while connecting to MySQL: {error}")

    finally:
        cursor.close()
        connection.close()

In [ ]:


def med_regular_greeting(phone_id):
  global username
  global usersex
  global userage
  global user_diseases
  global medication_alarm
  global injection_alarm
  global user_healthissues
  global sessiontime
  global context

  prompt = f"""The assistant greets the user based on the given user info and user health information.
              The assistant should talk to the elderly like a friendly neighbor.
              The assistant uses casual and informal conversation style.
              The assistant is talking to the elderly who wants to have friendly conversation. 
              The assistant should not use difficult words or phrases, and should be patient and understanding.
              The assistant speaks only Korean and is designed to help the elderly.

              The assistant should greet the user based on the given user personal information and user health information.
              if current time is within the user's medication time or injection time, the assistant should ask the user whether the user have a medicine or injection.
              - User information 
                username : {username}, user gdnder : {usersex}, user age : {userage}
              - User health information :
                The user's illnesses or diseases : {user_diseases}
                The medication that the user must take and the times the users need to take it, and assistant should ask the user whether the user have a medicine : {medication_alarm}
                The injection medication that the user must inject themselves and the times the users need to take it, and the assistant should ask the user whether the user have an injection : {injection_alarm}
                Current time : {sessiontime}
                If the current time is within 3 minutes of the user’s medication time or injection time, the assistant should briefly ask the user whether they have taken their medicine or injection, including a greeting message.
                The user's health issues about which the assistant sometimes ask a question : {user_healthissues}"""
      
  userhiddengreeting = [{"role": "user", "content": "안녕!" }]


  greetingresponse = openai_client.chat.completions.create(
      model="gpt-4o",
      messages=[{"role":"system", "content": prompt}] + userhiddengreeting,
      max_tokens=1024,
      temperature=0.5,
      stop=["\n"]
  )
  
  response_text = greetingresponse.choices[0].message.content
  conv_model = greetingresponse.model
  conversation_start = "med_regular_greeting"

  context.append({"role": "assistant", "content": response_text})
  
  save_context_to_db("assistant", response_text, conversation_start, conv_model)

  return response_text  




def get_greeting_from_summarization(phone_id): # first_greeting_tts(phone_id)
    global context
    global db_config
    # global context_counter
    # global sessiontime
    # global username
    
    try:
        connection = pymysql.connect(**db_config)
        cursor = connection.cursor()

        query = """SELECT next_first_question FROM summarization 
                    WHERE phone_id = %s AND session_id = (SELECT session_id
                                                        FROM summarization WHERE phone_id = %s
                                                        ORDER BY summ_created_at DESC
                                                        LIMIT 1
                                                        );"""
        values = (phone_id, phone_id)
        
        # 쿼리 실행
        cursor.execute(query, values)
        first_question_from_summ = cursor.fetchone()

        print("Selected next first question from summarization table.")

    except pymysql.Error as error:
        print(f"Error while connecting to MySQL: {error}")

    finally:
        cursor.close()
        connection.close()
        
    greetingresponse = "".join(first_question_from_summ)
    greeting_text = greetingresponse
    conv_model = "summarization"
    conversation_start = "get_greeting_from_summarization"

    context.append({"role": "assistant", "content": greeting_text})

    save_context_to_db("assistant", greeting_text, conversation_start, conv_model)
        
    # play_chatgpt_response_with_tts(greeting_text)
    return greeting_text
  


def make_weather_greeting(phone_id):
  global context
  global username
  global sessiontime
  
  temp_time = datetime.now(timezone('Asia/Seoul'))
  mod_datetime = temp_time - timedelta(hours = 0.5)
  input_date = mod_datetime.strftime('%Y%m%d%H%M')[:-4]
  input_time = mod_datetime.strftime('%Y%m%d%H%M')[-4:]

  nx = '73'
  ny = '134'
  serviceKey = "8SukIEKtVXQQ3NeKhumtUS3gof1NZbvkVyP6G6dzGAhc4kR8PHImAvQf3l5yadry8iDYX0MZf4MMvMIsm7hqoA%3D%3D"

  # url
  url = f"http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst?serviceKey={serviceKey}&numOfRows=60&pageNo=1&dataType=json&base_date={input_date}&base_time={input_time}&nx={nx}&ny={ny}"


  response = requests.get(url, verify=False)
  weather_result = json.loads(response.text)

  weather_result_append = []
  target_fcst_time = weather_result["response"]["body"]["items"]["item"][0]["fcstTime"]

  for item in weather_result["response"]["body"]["items"]["item"]:
      if item["fcstTime"] == target_fcst_time:
          weather_result_append.append(item)

  mod_result = []

  category_values = {}
  for item in weather_result_append:
      category_values[item['category']] = item['fcstValue']
      base_data = {
          'baseDate': item['baseDate'],
          'baseTime': item['baseTime'],
          'fcstDate': item['fcstDate'],
          'fcstTime': item['fcstTime'],
          'nx': item['nx'],
          'ny': item['ny']
      }

  mod_result.append({
      'baseDate': base_data['baseDate'],
      'baseTime': base_data['baseTime'],
      'category': category_values,
      'fcstDate': base_data['fcstDate'],
      'fcstTime': base_data['fcstTime'],
      'nx': base_data['nx'],
      'ny': base_data['ny']
  })

  lgt = mod_result[0]["category"]["LGT"]
  pty = mod_result[0]["category"]["PTY"]
  rn1 = mod_result[0]["category"]["RN1"]
  sky = mod_result[0]["category"]["SKY"]
  t1h = mod_result[0]["category"]["T1H"]
  wsd = mod_result[0]["category"]["WSD"]
  
  
  
  
  
  prompt = f"""The assistant greets the user based on the given user info and user health information.
              The assistant should talk to the elderly like a friendly neighbor.
              The assistant uses casual and informal greeting style.
              The assistant should not use difficult words or phrases.
              The assistant speaks only Korean and is designed to help the elderly.

              The assistant should greet the user based on the given current weather information.
              User information = username : {username}
              Weather information parameters :
                LGT means thunderstroke and the value means 에너지밀도(0.2~100kA/㎢) of thunderstroke
                PTY means precipitation type and the value means 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7)
                RN1 means 1시간 강수량(mm)
                SKY means sky condition and the value means 맑음(1), 구름조금(2), 구름많음(3), 흐림(4)
                T1H means temperature(°C)
                WSD means wind speed(m/s)
              - Current weather information 
                LGT : "{lgt}, PTY : {pty}, RN1 : {rn1}, SKY : {sky}, T1H : {t1h}, WSD : {wsd}
                
                The assistant should greet the user based on the given current weather information.
                The assistant should use short and concise greeting with weather information
                The assistant should ask the user's feeling in the greeting message.
                    ex) 안녕하세요. 이보람님. 오늘은 구름이 많이꼈네요. 날씨 때문에 우울하지는 않으세요?"""
                      
  userhiddengreeting = [{"role": "user", "content": "안녕!" }]


  greetingresponse = openai_client.chat.completions.create(
      model="gpt-4o",
      messages=[{"role":"system", "content": prompt}] + userhiddengreeting,
      max_tokens=1024,
      temperature=0.5,
      stop=["\n"]
  )

  response_text = greetingresponse.choices[0].message.content
  conv_model = greetingresponse.model
  conversation_start = "make_weather_greeting"
  context.append({"role": "assistant", "content": response_text})
  save_context_to_db("assistant", response_text, conversation_start, conv_model)

  return response_text  



In [ ]:



def get_previous_conversation(phone_id):
    try:
        connection = pymysql.connect(**db_config)
        cursor = connection.cursor()

        query = """SELECT session_id, conversation_start, role, content
        FROM context
        WHERE phone_id = %s AND session_id = (
            SELECT session_id
            FROM context WHERE phone_id = %s
            ORDER BY created_at DESC
            LIMIT 1
        );"""
        values = (phone_id, phone_id)

        cursor.execute(query, values)
        previous_conversation = cursor.fetchall()

        print("Previous conversation retrieved from the database.")

    except pymysql.Error as error:
        print(f"Error while connecting to MySQL: {error}")

    finally:
        cursor.close()
        connection.close()
    
    return previous_conversation

def make_summ_nextgreeting_from_chat(phone_id):
    global username
    previous_conversation = get_previous_conversation(phone_id)
    conversation_start = previous_conversation[0][1]
    current_time_summ = datetime.now(timezone('Asia/Seoul')).strftime('%Y-%m-%d %H:%M:%S')
    conv_session_id = previous_conversation[0][0]

    
    if len(previous_conversation) == 1 and previous_conversation[0][2] == "assistant":
        response_sum_text = 0
        response_greeting_text = 0
        summ_model = 0
    else:
        prompt_sum = """you have to summarize the previous conversation.
                        The summarization is consisted of only Korean.
                        The summarization should be short and concise."""


        request_summarization = f"""Summarize previous conversation given below
                                
                                user and assistant previous conversation:
                                {previous_conversation}"""

        
        context_sum = [{"role": "user", "content": request_summarization}]
        

        # ChatGPT-4o API
        response_sum = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role":"system", "content": prompt_sum}] + context_sum,
            max_tokens=1024,
            temperature=0.5,
            stop=["\n"]
        )
        
        
        
        prompt_greeting = """you have to give a first greeting question to start a new conversation, which is based on the summarization of the previous conversation.
                    The first greeting question is to talk to the elderly like a friendly neighbor.
                    The first greeting question uses casual and informal conversation style. 
                    The first greeting question should not use difficult words or phrases.
                    The first greeting question is consisted of only Korean.
                    
                    The first greeting question should be "short".
                    Remember to keep the conversation friendly and casual like a chat buddy.
                    """
        
        request_greeting = f"""Give me a appropriate greeting question to start next conversation.
                            The greeting question should be related to summarization that you made.
                            Additionally, the answer should start with a greeting and my nmae.
                                ex) 안녕하세요. 이보람님. [greeting question]
                                
                            My name : {username}
                            
                            Summarization of previous conversation:
                            {response_sum.choices[0].message.content}"""



        context_greeting = [{"role": "user", "content": request_greeting}]
        
        response_greeting = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role":"system", "content": prompt_greeting}] + context_greeting,
            max_tokens=1024,
            temperature=0.5,
            stop=["\n"]
        )
        
        response_sum_text = response_sum.choices[0].message.content
        response_greeting_text = response_greeting.choices[0].message.content
        
        summ_model = response_sum.model
    


    return response_sum_text, response_greeting_text, current_time_summ, conv_session_id, summ_model, conversation_start




def save_summarization_to_db(phone_id, username):
    response_sum_text, response_greeting_text, current_time_summ, conv_session_id, summ_model, conversation_start = make_summ_nextgreeting_from_chat(phone_id)
    # current_time = datetime.now(timezone('Asia/Seoul')).strftime('%Y-%m-%d %H:%M:%S')

    try:
        connection = pymysql.connect(**db_config)
        cursor = connection.cursor()

        query = "INSERT INTO summarization (session_id, summ_created_at, phone_id, username, conversation_start, summary_model, summary, next_first_question) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        values = (conv_session_id, current_time_summ, phone_id, username, conversation_start, summ_model, response_sum_text, response_greeting_text)

        cursor.execute(query, values)
        connection.commit()

        print("Summarization and next greeting saved to the database.")

    except pymysql.Error as error:
        print(f"Error while connecting to MySQL: {error}")

    finally:
        cursor.close()
        connection.close()



In [ ]:


def casual_greeting(phone_id, casual_situation):
    global context
    
    if casual_situation == 'morning':
        response = make_weather_greeting(phone_id)
    elif casual_situation == 'summarization':
        response = get_greeting_from_summarization(phone_id)
    elif casual_situation == 'evening':
        _, response_greeting, *_ = make_summ_nextgreeting_from_chat(phone_id)
        response = response_greeting
        conv_model = "script and summarization"
        conversation_start = "casual_greeting"
        context.append({"role": "assistant", "content": response})
        save_context_to_db("assistant", response, conversation_start, conv_model)
    return response


def med_reminding_greeting(phone_id):
    global context
    global username
    global current_timearea
    
    
    if current_timearea == '점심':
        prev_current_timearea = '아침'
    elif current_timearea == '저녁':
        prev_current_timearea = '점심'
    elif current_timearea in ['아침', '새벽']:
        prev_current_timearea = '어제'

    response = "{username}님, {prev_current_timearea}에 연락을 안받으셔서 걱정했어요. {prev_current_timearea}에 약 드시는 시간이었는데 드셨나요?"
    conv_model = "scripted"
    conversation_start = "med_reminding_greeting"
    
    context.append({"role": "assistant", "content": response})
    save_context_to_db("assistant", response, conversation_start, conv_model)
    
    return response




def get_casual_greeting(phone_id, current_timearea):

    if current_timearea == '아침':
        response = casual_greeting(phone_id,'morning')
    elif current_timearea == '점심':
        response = casual_greeting(phone_id,'summarization')
    elif current_timearea == '저녁':
        response = casual_greeting(phone_id,'evening')
    else:
        response = casual_greeting(phone_id,'summarization')
    return response



def get_greeting_response(phone_id, session_close_key, prev_summary, prev_conversation_start, current_timearea):
    if session_close_key == 'casual':
        if prev_summary == 0 :
            if prev_conversation_start == 'med_regular_greeting':
                resposne = med_reminding_greeting(phone_id)
            else:
                response = get_casual_greeting(phone_id, current_timearea)
        else:
                response = get_casual_greeting(phone_id, current_timearea)
    elif session_close_key in ['health_med_alarm', 'health_inj_alarm']:
        response = med_regular_greeting(phone_id)
    else:
        response = get_casual_greeting(phone_id, current_timearea)
    return response
    


In [ ]:



def chat_with_gpt(user_input, phone_id):
    global context
    global context_string
    global username
    global usersex
    global userage
    global user_diseases
    global medication_alarm
    global injection_alarm
    global user_healthissues


    temp_currenttime = datetime.now(timezone('Asia/Seoul')).time()
    
    # Iterate over each dictionary in the context list
    for i, entry in enumerate(context):
        # Format the string as 'role:content\n' (add '\n' if not the last item)
        if i < len(context) - 1:
            formatted_string = f"{entry['role']}:{entry['content']}\n"
        else:
            formatted_string = f"{entry['role']}:{entry['content']}"
        
        # Append the formatted string to the context_string list
        context_string.append(formatted_string)
    
    prompt = f"""The following is a friendly conversation between a human and an assistant.
                The assistant should talk to the elderly like a friendly neighbor.
                The assistant uses casual and informal conversation style.
                The assistant cares about the health, daily life, and family of the elderly.    
                The assistant is talking to the elderly who wants to have friendly conversation. 
                The assistant is to help the elderly relieve their depression or gloomy mood by having a conversation. 
                The assistant should not use difficult words or phrases, and should be patient and understanding.
                The assistant should give a question to the elderly to keep the conversation going.
                The assistant speaks only Korean and is designed to help the elderly.
                
                The assistant must respond "shortly" with no more than three sentences each time.
                Remember to keep the conversation friendly and casual like a chat buddy.
                Below is friendly and casual statement for the assistant to use.
                ex) 아, 아주머니 무릎 아프시다고 운동 못하신다고 하시던데, 저도 할머니가 그러셨거든요. 그래도 포기하지 마시고 수영이나 스트레칭 같은 가벼운 거라도 해보세요. 조금씩 하다 보면 아프지 않고 건강해지실 수 있을 거예요. 아주머니도 할 수 있다니까요, 파이팅!

                The assistant should understand and internalize the given user information and engage in conversation with the user based on this information.
                User information = username : {username}, user gdnder : {usersex}, user age : {userage}
                
                - User health information that the assistant can reference during the conversation:
                The user's illnesses or diseases : {user_diseases}
                The user's medication the user needs to take and the times they need to take it : {medication_alarm}
                The user's injection medication the user needs to take and the times they need to take it: {injection_alarm}
                Current time : {temp_currenttime}
                The user's other healthissues : {user_healthissues}
                
                Previous conversation:\n{context_string}\n\nuser: {user_input}\nassistant:"""
        
    context.append({"role": "user", "content": user_input})
    
    
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"system", "content": prompt}] + context,
        max_tokens=1024,
        temperature=0.5,
        stop=["\n"]
    )
    
    response_text = response.choices[0].message.content
    conv_model = response.model
    
    context.append({"role": "assistant", "content": response_text})
    
    userevenno = save_context_to_db("user", user_input)
    save_context_to_db("assistant", response_text, conv_model)
 
    return response_text, userevenno


In [43]:
def get_chatgpt_response(transcript):
    # Talk to the AI counselor via langchain 
    conversation_answer = chat_with_gpt(transcript, phone_id)
    
    # Return the AI's message content
    return conversation_answer

In [44]:


def play_chatgpt_response_with_tts(ai_response):
    # Generate speech from the GPT response using TTS

    response = openai_client.audio.speech.create(
        model="tts-1",          # Specifies the TTS model to use
        voice="alloy",          # Chooses a specific voice for the TTS
        input=ai_response      # The text input to be converted to speech
    )

    # # Stream the audio to a file
    # response.stream_to_file(speech_file_path)
    # # response.write_to_file(speech_file_path)
    # os.system("afplay " + speech_file_path)

    # # Remove the temporary speech file to clean up
    # os.remove(speech_file_path)
    
    return response

In [45]:
def save_wav(buffer, file_path):
    with wave.open(file_path, "wb") as wav_file: # "uploaded.wav"
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(16000)
        wav_file.writeframes(buffer)

def rename_wav_file(session_id, userevenno):
    # 기존 파일 경로
    humancvs_file_path = f"./{session_id}_.wav"
    
    # 새 파일 경로
    new_file_path = f"./{session_id}_{userevenno:04d}.wav"
    
    try:
        # 파일 이름 변경
        os.rename(humancvs_file_path, new_file_path)
        print(f"File renamed from {humancvs_file_path} to {new_file_path}")
    except FileNotFoundError:
        print(f"File {humancvs_file_path} not found.")
    except Exception as e:
        print(f"Error renaming file: {e}")
    
    return new_file_path

In [ ]:
def phoneid_db_update(phone_id, name):
    try:
      connection = pymysql.connect(**db_config)
      cursor = connection.cursor()
      
      # 트랜잭션 시작
      connection.begin()
      
      # update user table
      userupdatequery = """UPDATE user 
                            SET phone_id = %s WHERE username = %s;"""
      uuvalues = (phone_id, name)
      cursor.execute(userupdatequery, uuvalues)
      print("Update phone_id to user table.")
      
      
      # update alarm table
      alarmupdatequery = """UPDATE alarm
                            SET phone_id = %s WHERE username = %s;"""
      auvalues = (phone_id, name)
      cursor.execute(alarmupdatequery, auvalues)
      print("Update phone_id to alarm table.")
      
      
      # update healthinfo table
      healthinfoupdatequery = """UPDATE healthinfo
                                SET phone_id = %s WHERE username = %s;"""
      huvalues = (phone_id, name)
      cursor.execute(healthinfoupdatequery, huvalues)
      print("Update phone_id to healthinfo table.")
      
      connection.commit()
      return True

      
    except pymysql.Error as error:
      print(f"Error while connecting to MySQL: {error}")
      connection.rollback()
      return False

    finally:
      cursor.close()
      connection.close()

<!-- 복약지도 > 경각심 필요 / 노티 필요
>> 감독관이 지정한 질문 위주로 진행하게끔..

밥 > 아점저 시간에 하게 되면
복약
    혈압 > 시간지정 ex) 7~8시 사이에 물어보게되면
    당뇨 >
운동

건강 > 화재전환  -->


In [ ]:

app = FastAPI()
templates = Jinja2Templates(directory="templates")

@app.get("/client")
async def client(request: Request):
    return templates.TemplateResponse("client.html", {"request":request})

@app.websocket("/")
async def websocket_endpoint(websocket: WebSocket):

    global openai_client
    global context
    global context_string
    global context_counter
    global db_config

    print(f"client connected: {websocket.client}")
    await websocket.accept()
    await websocket.send_text("Welcome")
    locale.getpreferredencoding = lambda: "UTF-8" 
    
    phone_id = None
    user_info = None
    session = None

    while True:
           
        try:
            data = await websocket.receive_text()
            print(f"Received text data: {data}")
        
            if data.startswith("search#"):
                phone_id = data.split("#")[1]

                user_info_obj = UserInfo(db_config)
                user_info = user_info_obj.get_user_info(phone_id)
                username = user_info['username']
                usersex = user_info['usersex']
                userage = user_info['userage']
                user_diseases = user_info['disease']
                medication_alarm = user_info['health_med_alarm']
                injection_alarm = user_info['health_inj_alarm']
                user_healthissues = user_info['healthissue']
                prev_conversation_start = user_info['prev_conversation_start']
                prev_summary = user_info['prev_summary']

                session = UserSession(phone_id, user_info)
                session_id = session.session_id
                sessiontime = session.sessiontime
                session_close_alarm_time = session.close_time
                session_close_key = session.close_key
                current_timearea = session.current_timearea
                
                json_search_result = { "phone_id": phone_id, 
                                      "username": username, 
                                      "usersex": usersex, 
                                      "userage": userage,
                                      "user_diseases": user_diseases,
                                      "medication_alarm": medication_alarm,
                                      "injection_alarm": injection_alarm,
                                      "user_healthissues": user_healthissues}
                
                await websocket.send_text(f"search#{json.dumps(json_search_result,default=str, ensure_ascii = False)}")
            
            elif data.startswith("register#"):
                 name = data.split("#")[1]
                 if phoneid_db_update(phone_id, name):
                        await websocket.send_text("register#OK")
                 else:
                        await websocket.send_text("register#ERROR")
                #  if user_info and name == user_info['username']:
                #         await websocket.send_text("register#OK")
                #  else:
                #         await websocket.send_text("register#ERROR")
                            
            
            elif data.startswith("welcome_tts#"):
                if not session:
                    await websocket.send_text("Error: No session info")
                    continue
                greeting_text = get_greeting_response(phone_id, session_close_key, prev_summary, prev_conversation_start, current_timearea)
                await websocket.send_text(f"welcome_tts_text#{greeting_text}")
                
                
                greeting_file_path = f"./{session_id}_greeting.wav"
                play_chatgpt_response_with_tts(greeting_text).stream_to_file(greeting_file_path) # [phone_id]/[sessionid]_conv.wav
                # audio_files.append(greeting_file_path)
                with open(greeting_file_path, "rb") as audio_file:
                    greeting_wav = audio_file.read()
                    greeting_wav = base64.b64encode(greeting_wav) #.decode("utf-8")
                    await websocket.send_text(f"welcome_tts#{greeting_wav}")
                os.remove(greeting_file_path)
                                
            elif data.startswith("human_cvs#"):
                if not session:
                    await websocket.send_text("Error: No session info")
                    continue
                audio_data = data.split("#")[1]  # This would be the audio data in a real scenario
                audio_data = base64.b64decode(audio_data)
                humancvs_file_path = f"./{session_id}_.wav"
                save_wav(audio_data, humancvs_file_path)

                transcription_text = get_transcript(humancvs_file_path) # 'uploaded.wav'
                await websocket.send_text(f"human_cvs_text#{transcription_text}")

                llm_response, userevenno = chat_with_gpt(transcription_text, phone_id)
                await websocket.send_text(f"ai_cvs_text#{llm_response}")
                renamewavfile = rename_wav_file(session_id, userevenno)
                save_audiodir_to_context(renamewavfile, session_id, userevenno)
                
                response_file_path = f"./{session_id}_response.wav"
                play_chatgpt_response_with_tts(llm_response).stream_to_file(response_file_path)
                # audio_files.append(response_file_path)
                with open(response_file_path, "rb") as audio_file:
                    answer_wav = audio_file.read()
                    answer_wav = base64.b64encode(answer_wav)#.decode("utf-8")
                    await websocket.send_text(f"ai_cvs#{answer_wav}")
                os.remove(response_file_path)
        
        except WebSocketDisconnect:
            print("Websocket disconnected")
            # append_audio(greeting_file_path, audio_files)
            save_summarization_to_db(phone_id, username) #, greeting_file_path)
            print("Summarization saved to the database.")
            break
        
        except Exception as e:
            print(f"Error: {e}")
            await websocket.close()
            break
    
        
def run():
    import uvicorn
    uvicorn.run(app, host='0.0.0.0', port=8845)
        
if __name__ == "__main__":
    run()
    